In [2]:
import pandas as pd
import re

In [3]:
pd.set_option('display.max_columns', None)

In [44]:
compras = pd.read_csv('VendasGrano_csv.csv', encoding='latin1', delimiter=';', decimal=',', thousands='.')

In [45]:
print(compras.columns)


Index(['Data de Emissão', 'Data de Lançamento', 'Fornecedor',
       'Descrição Material', 'Qtde.', 'Unidade', 'Preço Unitário (R$)',
       'Acréscimo (R$)', 'Desconto (R$)', 'Total (R$)', 'Grupo do Material'],
      dtype='object')


In [61]:
def itens_semelhantes(compras, top_fornecedores):
    compras_fornecedores = compras[compras['Fornecedor'].isin(top_fornecedores)]
    produtos_por_fornecedor = compras_fornecedores.groupby(['Descrição Material', 'Fornecedor']).agg({'Qtde.': 'sum'}).reset_index()
    produtos_por_quantidade_fornecedores = produtos_por_fornecedor.groupby('Descrição Material').size().reset_index(name='Quantidade_Fornecedores')
    produtos_com_mais_de_um_fornecedor = produtos_por_quantidade_fornecedores[produtos_por_quantidade_fornecedores['Quantidade_Fornecedores'] > 1]['Descrição Material']

    return produtos_com_mais_de_um_fornecedor

def valores_por_fornecedor(compras, top_fornecedores, itens_semelhantes):
    # Obtendo produtos com mais de um fornecedor
    produtos_com_mais_de_um_fornecedor = itens_semelhantes(compras, top_fornecedores)

    # Iterando sobre os produtos
    for produto in produtos_com_mais_de_um_fornecedor:
        print('Produto:', produto)
        # Agrupando por fornecedor
        fornecedores_produto = compras[compras['Descrição Material'] == produto].groupby('Fornecedor').agg({'Total (R$)': 'sum', 'Preço Unitário (R$)': 'mean', 'Qtde.': 'sum', 'Unidade': 'first'}).reset_index()
        fornecedores_produto.columns = ['Fornecedor', 'Total Gasto', 'Valor Unitário Médio (R$)', 'Quantidade', 'Unidade']
        for index, row in fornecedores_produto.iterrows():
            fornecedor = row['Fornecedor']
            total_gasto = row['Total Gasto']
            quantidade = row['Quantidade']
            valor_unitario_medio = row['Valor Unitário Médio (R$)']
            unidade_comprada = row['Unidade']
            # Formatando os números para exibir com duas casas decimais
            total_gasto_formatted = '{:.2f}'.format(total_gasto)
            valor_unitario_medio_formatted = '{:.2f}'.format(valor_unitario_medio)
            print(f'Fornecedor: {fornecedor}')
            print(f'Quantidade de itens comprados: {quantidade}')
            print(f'Total gasto: {total_gasto_formatted}')
            print(f'Valor Unitário Médio (R$): {valor_unitario_medio_formatted}')
            print(f'Unidade Comprada: {unidade_comprada}')
        print()

top_fornecedores = ['UMEHARA ALIMENTOS LTDA', 'Hortifruti Genesis Ltda', 'Supermercados unidos (pai da luiza)']
valores_por_fornecedor(compras, top_fornecedores, itens_semelhantes)



Produto: Abacate
Fornecedor: Hortifruti Genesis Ltda
Quantidade de itens comprados: 27.0
Total gasto: 189.00
Valor Unitário Médio (R$): 7.00
Unidade Comprada: UN    
Fornecedor: Supermercados unidos (pai da luiza)
Quantidade de itens comprados: 37.83
Total gasto: 166.72
Valor Unitário Médio (R$): 4.36
Unidade Comprada: UN    
Fornecedor: UMEHARA ALIMENTOS LTDA
Quantidade de itens comprados: 17.0
Total gasto: 117.12
Valor Unitário Médio (R$): 6.89
Unidade Comprada: UN    

Produto: Abacaxi
Fornecedor: Hortifruti Genesis Ltda
Quantidade de itens comprados: 39.0
Total gasto: 342.50
Valor Unitário Médio (R$): 8.85
Unidade Comprada: UN    
Fornecedor: UMEHARA ALIMENTOS LTDA
Quantidade de itens comprados: 117.0
Total gasto: 1115.41
Valor Unitário Médio (R$): 9.68
Unidade Comprada: UN    

Produto: Abobora sergipana
Fornecedor: Hortifruti Genesis Ltda
Quantidade de itens comprados: 91.0
Total gasto: 660.31
Valor Unitário Médio (R$): 7.36
Unidade Comprada: KG    
Fornecedor: UMEHARA ALIMENTOS 